In [14]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
technical_indicators_scaled = pd.read_csv('/content/drive/My Drive/technical_indicators_scaled.csv', index_col=0)
ticker_data = pd.read_csv('/content/drive/My Drive/ticker_data.csv', index_col=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Define a function to label the data based on the fixed number of hours (d)
def label_data_fixed_days(row_index, d, close_prices, binary=True):
    if row_index + d >= len(close_prices):
        return None
    current_price = close_prices.iloc[row_index]
    future_price = close_prices.iloc[row_index + d]
    if binary:
        return 1 if future_price > current_price else 0
    else:
        return 1 if future_price > current_price else -1


In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report

d_values = list(range(1, 15))  # Test different values of d
best_d, best_model, best_accuracy, best_report = None, None, 0, None

for d in d_values:
    # Label the data using the current value of d
    technical_indicators_scaled['Label'] = 0
    for idx in range(len(technical_indicators_scaled)):
        binary_label = label_data_fixed_days(idx, d, ticker_data['Close'], binary=True)
        if binary_label is not None:
            technical_indicators_scaled.loc[technical_indicators_scaled.index[idx], 'Label'] = binary_label

    y = technical_indicators_scaled['Label'].values
    X = technical_indicators_scaled.drop(columns=['Label'])

    # Remove the last d rows
    X = X.iloc[:-d]
    y = y[:-d]

    # Reshape the input data into a 3D array as required by LSTM
    n_features = X.shape[1]
    X = X.to_numpy().reshape(-1, 1, n_features)

    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(units=150, activation='relu', input_shape=(1, n_features)))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    tscv = TimeSeriesSplit(n_splits=5)
    accuracy_scores = []
    y_true_all = []
    y_pred_all = []

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train the model
        model.fit(X_train, y_train, epochs=400, batch_size=128, verbose=0)

        # Evaluate the model
        y_pred = (model.predict(X_test) > 0.5).astype("int32")
        accuracy = accuracy_score(y_test, y_pred)
        accuracy_scores.append(accuracy)

        y_true_all.extend(y_test)
        y_pred_all.extend(y_pred.flatten())

    # Calculate the average accuracy
    average_accuracy = np.mean(accuracy_scores)

    if average_accuracy > best_accuracy:
        best_d = d
        best_accuracy = average_accuracy
        best_model = model
        best_report = classification_report(y_true_all, y_pred_all)

    print(f"d = {d}, Average accuracy: {average_accuracy}")

print(f"Best d value: {best_d}, with average accuracy: {best_accuracy}")
print("\nClassification report:")
print(best_report)


KeyboardInterrupt: ignored

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
